In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# code adapted from 
# https://keras.io/examples/mnist_cnn/
# http://aqibsaeed.github.io/2017-08-11-genetic-algorithm-for-optimizing-rnn/
# https://github.com/aqibsaeed/Genetic-Algorithm-RNN/blob/master/Genetic-Algorithm-RNN.ipynb

Using TensorFlow backend.


In [65]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

np.random.seed(1120)

In [64]:
batch_size = 128
num_classes = 10
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(_x_train, _y_train), (_x_test, _y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    _x_train = _x_train.reshape(_x_train.shape[0], 1, img_rows, img_cols)
    _x_test = _x_test.reshape(_x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    _x_train = _x_train.reshape(_x_train.shape[0], img_rows, img_cols, 1)
    _x_test = _x_test.reshape(_x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = _x_train[:1000,:]
y_train = _y_train[:1000]
x_val = _x_train[1000:1200,:]
y_val = _y_train[1000:1200]
x_test = _x_test[:1000,:]
y_test = _y_test[:1000]


x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_val /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])

print('Test accuracy:', score[1])

x_train shape: (1000, 28, 28, 1)
1000 train samples
1000 test samples
Train on 1000 samples, validate on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 3s 3ms/step - loss: 1.9507 - acc: 0.3560 - val_loss: 1.3835 - val_acc: 0.5360
Epoch 2/5
1000/1000 [==============================] - 2s 2ms/step - loss: 1.2116 - acc: 0.6170 - val_loss: 0.9379 - val_acc: 0.7260
Epoch 3/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.7780 - acc: 0.7620 - val_loss: 0.7176 - val_acc: 0.7920
Epoch 4/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.5482 - acc: 0.8430 - val_loss: 0.4492 - val_acc: 0.8570
Epoch 5/5
1000/1000 [==============================] - 2s 2ms/step - loss: 0.4160 - acc: 0.8710 - val_loss: 0.5933 - val_acc: 0.7980
Test loss: 0.5932987847328186
Test accuracy: 0.798


In [77]:
def train_evaluate(ga_individual_solution):
    # Decode GA solution to integer for window_size and num_units
    window_size_bits = BitArray(ga_individual_solution[:3])
    num_units_bits0 = BitArray(ga_individual_solution[3:6])
    num_units_bits1 = BitArray(ga_individual_solution[6:])
    window_size = window_size_bits.uint
    num_units_l0 = num_units_bits0.uint
    num_units_l1 = num_units_bits1.uint
    print('\nWindow Size: ', window_size, ', Num of Units: ', num_units_l0,num_units_l1)
    min_win = 2
    min_unit = 3
    # Return fitness score of 100 if window_size or num_unit is zero
    if any([window_size > 10,
            window_size <= min_win,
            num_units_l0 > 1000, 
            num_units_l0 < min_unit,
            num_units_l1 > 1000, 
            num_units_l1 < min_unit]):
        return 100, 
    
    model = Sequential()
    model.add(Conv2D(num_units_l0, kernel_size=(window_size,window_size),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(num_units_l1, (window_size,window_size), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_val, y_val))
    score = model.evaluate(x_val, y_val, verbose=0)
    
    y_pred = model.predict(x_val)
    
    # Calculate the RMSE score as fitness score for GA (INAPPRORIATE METRIC!)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print('Validation RMSE: ', rmse,'\n')
    
    return rmse,

In [78]:
population_size = 4
num_generations = 10
gene_length = 9

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.6)
toolbox.register('individual', tools.initRepeat, creator.Individual, 
                 toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
    ngen = num_generations, verbose = False)

c:\python37\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
c:\python37\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)



Window Size:  7 , Num of Units:  6 7
Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [==============================] - 3s 3ms/step - loss: 2.2677 - acc: 0.1390 - val_loss: 2.1476 - val_acc: 0.3950
Epoch 2/5
1000/1000 [==============================] - 1s 640us/step - loss: 1.8943 - acc: 0.3670 - val_loss: 1.3678 - val_acc: 0.5600
Epoch 3/5
1000/1000 [==============================] - 1s 590us/step - loss: 1.3552 - acc: 0.5470 - val_loss: 0.8633 - val_acc: 0.7550
Epoch 4/5
1000/1000 [==============================] - 1s 614us/step - loss: 0.9268 - acc: 0.6930 - val_loss: 0.7189 - val_acc: 0.7750
Epoch 5/5
1000/1000 [==============================] - 1s 603us/step - loss: 0.7259 - acc: 0.7560 - val_loss: 0.5886 - val_acc: 0.8000
Validation RMSE:  0.16362362 


Window Size:  2 , Num of Units:  7 0

Window Size:  3 , Num of Units:  2 6

Window Size:  7 , Num of Units:  6 5
Train on 1000 samples, validate on 200 samples
Epoch 1/5
1000/1000 [=============================

In [82]:
# Print top N solutions - (1st only, for now)
best_individuals = tools.selBest(population,k = 1)
best_window_size = None
best_num_units = None

for bi in best_individuals:
    window_size_bits = BitArray(bi[:3])
    num_units_bits0 = BitArray(bi[3:6])
    num_units_bits1 = BitArray(bi[6:])
    window_size = window_size_bits.uint
    num_units_l0 = num_units_bits0.uint
    num_units_l1 = num_units_bits1.uint
    print('\nWindow Size: ', window_size, ', Num of Units: ', num_units_l0,num_units_l1)


Window Size:  3 , Num of Units:  4 4


In [80]:
best_individuals

[[0, 1, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0]]

In [81]:
population

[[0, 1, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0]]